In [ ]:
# matplotlib 한글 깨짐 해결
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 이 셀 실행 후 런터임 재실행을 실시하세요

In [ ]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
import random
import datetime
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') # 한글설정

In [ ]:
# 파이썬 지수를 정수로 표현!
pd.options.display.float_format = '{:.5f}'.format

# 코랩 그래프 크게!

%matplotlib inline

plt.rcParams['figure.figsize'] = (40, 40)

In [ ]:

# 한글설정
plt.rc('font', family='NanumBarunGothic')

# 파이썬 지수를 정수로 표현
pd.options.display.float_format = '{:.5f}'.format

# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# 데이터 불러오기

test = pd.read_csv('/content/drive/My Drive/data/bigcorn_data/2019test_(3).csv', encoding = 'cp949')
train = pd.read_csv('/content/drive/My Drive/data/bigcorn_data/2019train_(3).csv', encoding = 'cp949')



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

print('test', test.columns)
print('train', train.columns)

print(test.shape)
print(train.shape)

In [ ]:
def get_stacking_data(model, X_train, y_train, X_test):
    from sklearn.model_selection import KFold
    # 지정된 n_folds값으로 kfold 생성
    kfold = KFold(n_splits=3,random_state=0)

    # 추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_predict = np.zeros((X_train.shape[0],1))
    test_predict = np.zeros((X_test.shape[0], 3))
    print(type(test_predict))
    print(type(test_predict))

    print("model:", model.__class__.__name__)

    for cnt, (train_index, valid_index) in enumerate(kfold.split(X_train)) :

        X_tr  = X_train.iloc[train_index]
        y_tr = y_train.iloc[train_index]
        X_te = X_train.iloc[valid_index]
        model.fit( X_tr, y_tr)
    # # 폴더 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장
        train_fold_predict[valid_index, :]=model.predict(X_te).reshape(-1,1)

    # # 입력된 원본 데스트 데이터를 폴드 세트 내 학습된 기반 모델에서 예측 후 데이터 저장
        test_predict[:, cnt] = model.predict(X_test).reshape(1,-1)

    test_predict_mean = np.mean(test_predict, axis=1).reshape(-1,1)

  # train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean 테스트 데이터
    return train_fold_predict, test_predict_mean                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
# MAPE 구해주는 함수
def func_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

## 취급액boxcox > 취급액변환
def original_target(boxcox):
    return (0.2222222 * boxcox +1)**(1/0.2222222)

# 각각의 Valdation set마다 PCA를 적용해 주고, cross validation까지 수행해주는 함수
def PCA_CV(K, train, params, all_feature, pca_feature, target, onehot_feature):
    all_mape = []
    cnt = 1
    np.random.seed(1234)
    kfold = KFold(n_splits=K, shuffle=False)
    for t, v in kfold.split(train):
        # train / val > index 부여
        train_cv = train.iloc[t]
        val_cv = train.iloc[v]

        # train : x, y 나누기
        x_train = train_cv.loc[:, all_feature]
        y_train = train_cv.loc[:, target]
        y_train = y_train.iloc[:, 0]

        # val : x, y 나누기
        x_val = val_cv.loc[:, all_feature]
        y_val = val_cv.loc[:, target]
        y_val = y_val.iloc[:, 0]
        

        if len(pca_feature) != 0 :

            # PCA를 진행할 칼럼만 추출
            x_train_PCA = x_train.loc[:, pca_feature]
            remain_x_train = x_train.drop(pca_feature, axis=1, inplace = False).reset_index(drop = True)
        
            x_val_PCA = x_val.loc[:, pca_feature]
            remain_x_val = x_val.drop(pca_feature, axis=1, inplace = False).reset_index(drop = True)
            
            # PCA를 위한 Z-정규화
            m = np.mean(x_train_PCA)
            s = np.std(x_train_PCA)
            standard_x_train_PCA = (x_train_PCA - m) / s
            standard_x_val_PCA = (x_val_PCA - m) / s

            
            # PCA진행
            # 1. train_set
            pca1 = PCA(n_components=5)    ########################################### 임의로 PCA의 수를 지정해 줄 수 있어요
            pca_x_train = pca1.fit_transform(standard_x_train_PCA)
            pca_x_train_df = pd.DataFrame(data = pca_x_train
                    , columns = ['pca1', 'pca2', 'pca3'])
            # 2. val_set
            pca2 = PCA(n_components=5)    ########################################### 임의로 PCA의 수를 지정해 줄 수 있어요 (위에랑 같은 숫자로 지정해 주어야 해요 ㅠ)
            pca_x_val = pca2.fit_transform(standard_x_val_PCA)
            pca_x_val_df = pd.DataFrame(data = pca_x_val
                    , columns = ['pca1', 'pca2', 'pca3'])
            

            PCA_x_train = pd.concat([remain_x_train, pca_x_train_df],  axis = 1, sort=False)
            PCA_x_val = pd.concat([remain_x_val, pca_x_val_df],  axis = 1, sort=False)

            # OneHot Encoding
            all_data  = PCA_x_train.append(PCA_x_val)
            onehot_all_data = pd.get_dummies(data = all_data, columns=onehot_feature, drop_first = False)

            x_train = onehot_all_data.iloc[:len(PCA_x_train), ]
            x_val = onehot_all_data.iloc[len(PCA_x_train):, ]


        else :
            all_data = x_train.append(x_val)
            onehot_all_data = pd.get_dummies(data = all_data, columns=onehot_feature, drop_first = False)

            x_train = onehot_all_data.iloc[0:len(x_train), ]
            x_val = onehot_all_data.iloc[len(x_train):len(onehot_all_data), ]

        ############################################################# 변수 전처리 완료!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

        # 모델
        # model = lgb.LGBMRegressor(learning_rate = params['learning_rate'],
        #                           num_iterations = params['num_iterations'],
        #                           max_depth = params['max_depth'],
        #                           boosting  = 'dart',
        #                           eval_metrics = 'mape')
        # model.fit(x_train, y_train)

        # y_pred = model.predict(x_val)




        # 모델링넣는부분 
        lgbm_reg = lgb.LGBMRegressor(learning_rate = 0.1,
                              num_iterations = 1000,
                              max_depth = 6,
                              feature_fraction = 0.7,
                              boosting  = 'dart',
                              eval_metrics = 'mape')

        xgb_reg = XGBRegressor(learning_rate=0.03,max_depth_list=6,min_child_weight=1,gamma=0.0)
        hgb = HistGradientBoostingRegressor(max_leaf_nodes=31,learning_rate=0.05,min_samples_leaf=26,max_iter=120)


        #모델 추가 :

        # 모델_train , 모델_test = get_stacking_data(모델, x_train, y_train,x_val) 
        hgb_train, hgb_test = get_stacking_data(hgb, x_train, y_train,x_val)
        xgb_reg_train, xgb_reg_test = get_stacking_data(xgb_reg, x_train, y_train,x_val)
        lgbm_reg_train, lgbm_reg_test = get_stacking_data(lgbm_reg, x_train, y_train,x_val)

        #                   = np.concatenate(( hgb_train, xgb_reg_train, lgbm_reg_train , 딥러닝_Train), axis=1)
        Stack_final_X_train = np.concatenate(( hgb_train, xgb_reg_train, lgbm_reg_train), axis=1)

        #                   = np.concatenate(( hgb_train, xgb_reg_train, lgbm_reg_train , 딥러닝_test), axis=1)
        Stack_final_X_test = np.concatenate((hgb_test, xgb_reg_test, lgbm_reg_test), axis=1)
        lgbm_reg_train.fit(Stack_final_X_train, y_train)
        final = lgbm_reg_train.predict(Stack_final_X_test)

        # model =  RandomForestRegressor(random_state=0)
        # model.fit(x_train, y_train)
        # y_pred = model.predict(x_val)


        if target[0] == '취급액' :
            mape = func_mape(y_val, final)
            print(cnt, '-Fold MAPE : ' ,mape)
            all_mape.append(mape)
            cnt += 1
        else:
            mape = func_mape(original_target(y_val), original_target(final))
            print(cnt, '-Fold MAPE : ' ,mape)
            all_mape.append(mape)
            cnt += 1
    

    print('---------------------------------')
    print('3-Fold Mean MAPE : ', np.mean(all_mape))
    print('모든 mape : ', all_mape)
    print(' ')

    # 변수중요도 플랏
    # df = {'feature' : pd.Series(x_train.columns), 'importance' : pd.Series(model.feature_importances_)}
    # importance_df = pd.DataFrame(df)
    # importance_df = importance_df.sort_values(by = ['importance'])
    # importance_df.plot.barh(y = 'importance', x = 'feature')

    return final, y_val, np.mean(all_mape),xgb_reg, Stack_final_X_train

In [ ]:
# 실행해서 우리 데이터에 어떤 칼럼이 존재하는지 보세요
# 'Unnamed: 0'은 index번호라서 무조건 뺴면되요~

print('train', train.columns)

In [ ]:
# K-Fold 지정
K = 3

# 사용하고자 하는 *모든* 변수를 입력하세요
all_feature = ['노출(분)',
'판매단가',
'월',
'일',
'168시간',
'매진여부',
'count',
'한달_상품군빈도',
'한달_중분류빈도',
'한달_소분류빈도',
'한달_그룹코드빈도',
'기온',
'강수량',
'풍속',
'습도',
'적설',
'전운량',
'미세먼지',
'초미세먼지',
'판매단가순위',
'상품군판매단가순위',
'상품군요일순위',
'상품군24시간순위',
'중분류요일순위',
'중분류24시간순위',
'상품군168시간순위',
'중분류168시간순위',
'판매단가rank',
'시간순위',
'시간순위168',
'브랜드순위',
'중분류순위',
'소분류순위',
'방송내_순서',
'방송순서',
'판매상품종류수',
'판매상품종류비율',
'그룹코드_전체횟수',
'그룹코드_쪽박횟수',
'그룹코드_대박횟수',
'그룹코드_쪽박확률',
'그룹코드_대박확률',
'브랜드별_전체횟수',
'브랜드별_대박횟수',
'브랜드별_대박확률',
'중분류별_전체횟수',
'중분류별_대박횟수',
'중분류별_대박확률',
'소분류별_전체횟수',
'소분류별_대박횟수',
'소분류별_대박확률',
'168시간_COS',
'168시간_SIN',
'24시간_COS',
'24시간_SIN',
'방송내_상품종류별_점수',
'전체편성비율',
'방송내_상품선호도',
'결제수단']

# PCA를 수행하고자 하는 변수를 입력하세요
pca_feature = []

# Target 변수를 입력하세요 '취급액'으로 넣어도 됩니다.
target = ['취급액boxcox']

# all_feature에 입력했던 범주형 변수를 입력하세요
onehot_feature = ['결제수단']
# 파라매터를 입력하세요
params = {'learning_rate' : 0.1, 'num_iterations' : 1000, 'max_depth' : 6}

In [ ]:
# Grid search 없이 cross validation만 수행

y_pred, y_real, temp,model,X = PCA_CV(K, train, params, all_feature, pca_feature, target, onehot_feature)


In [ ]:
print(y_pred)
print(y_real)

In [ ]:
# Grid search 없이 cross validation만 수행

y_pred, y_real, temp,model,X = PCA_CV(K, train, params, all_feature, pca_feature, target, onehot_feature)


In [ ]:
pd.Series(list(X))[0]

In [ ]:
imp = model.feature_importances_
feature_list = pd.concat([pd.Series(list(X)), pd.Series(model.feature_importances_)], axis=1)
feature_list.columns = ['features_name', 'importance']
feature_list.sort_values("importance", ascending =False)[:50]

In [ ]:
imp = model.feature_importances_
feature_list = pd.concat([pd.Series(list(X)), pd.Series(model.feature_importances_)], axis=1)
feature_list.columns = ['features_name', 'importance']
feature_list.sort_values("importance", ascending =True)[:50]

In [ ]:
# Grid search 없이 cross validation만 수행

y_pred, y_real, temp = PCA_CV(K, train, params, all_feature, pca_feature, target, onehot_feature)


In [ ]:
# Grid search 없이 cross validation만 수행

y_pred, y_real, temp = PCA_CV(K, train, params, all_feature, pca_feature, target, onehot_feature)


In [ ]:
a = original_target(y_real)
b = original_target(y_pred)



In [ ]:
# 실제 Target과 예측 Target의 그래프 추이
plt.rcParams['figure.figsize'] = (20, 5)

data = {
    'real' : y_real,
    'pred' : y_pred
}


data
df = pd.DataFrame(data)

df.plot.line(linewidth = 0.5)

In [ ]:
# 사용하고자 하는 *모든* 변수를 입력하세요
all_feature = ['노출(분)',
'판매단가',
'월',
'일',
'168시간',
'매진여부',
'count',
'한달_상품군빈도',
'한달_중분류빈도',
'한달_소분류빈도',
'한달_그룹코드빈도',
'기온',
'강수량',
'풍속',
'습도',
'적설',
'전운량',
'미세먼지',
'초미세먼지',
'판매단가순위',
'상품군판매단가순위',
'상품군요일순위',
'상품군24시간순위',
'중분류요일순위',
'중분류24시간순위',
'상품군168시간순위',
'중분류168시간순위',
'판매단가rank',
'시간순위',
'시간순위168',
'브랜드순위',
'중분류순위',
'소분류순위',
'방송내_순서',
'방송순서',
'판매상품종류수',
'판매상품종류비율',
'그룹코드_전체횟수',
'그룹코드_쪽박횟수',
'그룹코드_대박횟수',
'그룹코드_쪽박확률',
'그룹코드_대박확률',
'브랜드별_전체횟수',
'브랜드별_대박횟수',
'브랜드별_대박확률',
'중분류별_전체횟수',
'중분류별_대박횟수',
'중분류별_대박확률',
'소분류별_전체횟수',
'소분류별_대박횟수',
'소분류별_대박확률',
'168시간_COS',
'168시간_SIN',
'24시간_COS',
'24시간_SIN',
'방송내_상품종류별_점수',
'최소판매수량',
'전체편성비율',
'방송내_상품선호도',
'결제수단']

# PCA를 수행하고자 하는 변수를 입력하세요
pca_feature = []

# Target 변수를 입력하세요 '취급액'으로 넣어도 됩니다.
target = ['취급액boxcox']

# all_feature에 입력했던 범주형 변수를 입력하세요
onehot_feature = ['결제수단']

In [ ]:
# Gridsearch 구현

K = 3
# all_feature = []  # 사용하는 변수를 다 집어 넣으세요
# pca_feature = []  # PCA를 진행하고자 하는 변수를 집어 넣으세요
# target = ['취급액boxcox'] # 타겟변수를 집어 넣으세요 / '취급액' or '취급액boxcox' 
# onehot_feature = [] # 범주형 변수를 집어 넣으세요





np.random.seed(1234)
min = 99999
for learning_rate in [0.1, 0.2, 0.3]:  # learning_rate
    for num_iterations in [1000, 1200]: #num_iterations
        for max_depth in [6, 8] : # max_depth
            for feature_fraction in [0.7, 0.8]:
                params = {'learning_rate' : learning_rate, 'num_iterations' : num_iterations, 'max_depth' : max_depth, 'feature_fraction' : feature_fraction}
                y_pred, y_real, temp = PCA_CV(K, train, params, all_feature, pca_feature, target, onehot_feature)
                if min > temp :
                    min = temp
                    best_param = params
print()
print()
print()
print('가장 낮은 MAPE : ', min) # 가장 낮은 MAPE를 출력합니다
print('Best parametor :', best_param) # MAPE가 가장 낮을때의 파라매터를 출력합니다


In [ ]:
print(min)
print(best_param)


In [ ]:
kfold = KFold(n_splits=3, shuffle=False)

for a, b in kfold.split(train):
    print('train : ', a)
    print('test :', b)

In [ ]:
### 최종예측!


# MAPE 구해주는 함수
def func_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

### Final Predict
def final_predict(train, test, params, all_feature, pca_feature, target, onehot_feature):
    # train : x, y 나누기
    np.random.seed(1234)
    x_train = train.loc[:, all_feature]
    y_train = train.loc[:, target]
    y_train = y_train.iloc[:, 0]

    # test : x, y 나누기
    x_test = test.loc[:, all_feature]
    y_test = test.loc[:, target]
    y_test = y_test.iloc[:, 0]

    if len(pca_feature) != 0 :
        # PCA를 진행할 칼럼만 추출
        x_train_PCA = x_train.loc[:, pca_feature]
        remain_x_train = x_train.drop(pca_feature, axis=1, inplace = False).reset_index(drop = True)
        
        x_val_PCA = x_test.loc[:, pca_feature]
        remain_x_test = x_test.drop(pca_feature, axis=1, inplace = False).reset_index(drop = True)
            
            # PCA를 위한 Z-정규화
        m = np.mean(x_train_PCA)
        s = np.std(x_train_PCA)
        standard_x_train_PCA = (x_train_PCA - m) / s
        standard_x_test_PCA = (x_test_PCA - m) / s

            
        # PCA진행
        # 1. train_set
        pca1 = PCA(n_components=5)    ########################################### 임의로 PCA의 수를 지정해 줄 수 있어요
        pca_x_train = pca1.fit_transform(standard_x_train_PCA)
        pca_x_train_df = pd.DataFrame(data = pca_x_train
                , columns = ['pca1', 'pca2', 'pca3', 'pca4', 'pca5'])
        # 2. val_set
        pca2 = PCA(n_components=5)    ########################################### 임의로 PCA의 수를 지정해 줄 수 있어요 (위에랑 같은 숫자로 지정해 주어야 해요 ㅠ)
        pca_x_test = pca2.fit_transform(standard_x_test_PCA)
        pca_x_test_df = pd.DataFrame(data = pca_x_val
                , columns = ['pca1', 'pca2', 'pca3', 'pca4', 'pca5'])
            

        PCA_x_train = pd.concat([remain_x_train, pca_x_train_df],  axis = 1, sort=False)
        PCA_x_val = pd.concat([remain_x_val, pca_x_val_df],  axis = 1, sort=False)

        # OneHot Encoding
        all_data  = PCA_x_train.append(PCA_x_test)
        onehot_all_data = pd.get_dummies(data = all_data, columns=onehot_feature, drop_first = False)

        x_train = onehot_all_data.iloc[:len(PCA_x_train), ]
        x_test = onehot_all_data.iloc[len(PCA_x_train):, ]        

    else:
        all_data = x_train.append(x_test)
        onehot_all_data = pd.get_dummies(data = all_data, columns=onehot_feature, drop_first = False)

        x_train = onehot_all_data.iloc[0:len(x_train), ]
        x_test = onehot_all_data.iloc[len(x_train):len(onehot_all_data), ]        

    ## 예측시작
    np.random.seed(1234)
    # model = lgb.LGBMRegressor(learning_rate = params['learning_rate'],
    #                           num_iterations = params['num_iterations'],
    #                           max_depth = params['max_depth'],
    #                           boosting  = 'dart',
    #                           eval_metrics = 'mape')
   

    lgbm_reg = lgb.LGBMRegressor(learning_rate = 0.1,
                              num_iterations = 1000,
                              max_depth = 6,
                              feature_fraction = 0.7,
                              boosting  = 'dart',
                              eval_metrics = 'mape')

    xgb_reg = XGBRegressor(learning_rate=0.03,max_depth_list=6,min_child_weight=1,gamma=0.0)
    hgb = HistGradientBoostingRegressor(max_leaf_nodes=31,learning_rate=0.05,min_samples_leaf=26,max_iter=120)

    hgb_train, hgb_test = get_stacking_data(hgb, x_train, y_train,x_test)
    xgb_reg_train, xgb_reg_test = get_stacking_data(xgb_reg, x_train, y_train,x_test)
    lgbm_reg_train, lgbm_reg_test = get_stacking_data(lgbm_reg, x_train, y_train,x_test)

    Stack_final_X_train = np.concatenate(( hgb_train, xgb_reg_train, lgbm_reg_train), axis=1)
    Stack_final_X_test = np.concatenate((hgb_test, xgb_reg_test, lgbm_reg_test), axis=1)
    xgb_reg.fit(Stack_final_X_train, y_train)
    final = xgb_reg.predict(Stack_final_X_test)



    if target[0] == '취급액':
        mape = func_mape(y_test, final)
        print('Final MAPE : ' ,mape)
        y_pred = y_pred
        y_test = np.array(y_test)
        mape = mape

    else:
        final = original_target(final)
        y_test = original_target(np.array(y_test))
        mape = func_mape(y_test, final)
        print('Final MAPE : ' ,mape)
    
    # 변수중요도 플랏
    df = {'feature' : pd.Series(x_train.columns), 'importance' : pd.Series(model.feature_importances_)}
    importance_df = pd.DataFrame(df)
    importance_df = importance_df.sort_values(by = ['importance'])
    importance_df.plot.barh(y = 'importance', x = 'feature')

    return y_pred, y_test, mape

In [ ]:
# # K-Fold 지정
# K = 3

# # 사용하고자 하는 *모든* 변수를 입력하세요
# all_feature = []

# # PCA를 수행하고자 하는 변수를 입력하세요
# pca_feature = []

# # Target 변수를 입력하세요 '취급액'으로 넣어도 됩니다.
# target = []

# # all_feature에 입력했던 범주형 변수를 입력하세요
# onehot_feature = []


# params = {'learning_rate' : 0.1, 'num_iterations' : 1000, 'max_depth' : 6}

In [ ]:
# test data : 2019.06 예측

y_pred, y_real, mape = final_predict(train, test, params, all_feature, pca_feature, target, onehot_feature)

In [ ]:
print(y_pred)
print(y_real)

In [ ]:
print(y_pred)
print(y_real)

In [ ]:
# 실제 Target과 예측 Target의 그래프 추이
plt.rcParams['figure.figsize'] = (200, 40)

data = {
    'real' : y_real,
    'pred' : y_pred
}


data
df = pd.DataFrame(data)

df.plot.line(linewidth = 1)

In [ ]:
data = {
    'real' : y_real,
    'pred' : y_pred,
    'MAPE' : abs( ( y_real - y_pred ) / y_real ) * 100
}



df = pd.DataFrame(data)

newdf = pd.concat([test.loc[:, ['방송일시', '상품명', '상품군']], df], axis = 1, sort=False)
n_df = newdf.sort_values('MAPE')

n_df.to_csv("/content/drive/My Drive/data/bigcorn_data/제발되어라sdfsdf.csv")


In [ ]:
d1 = pd.DataFrame(train["그룹코드"])  

In [ ]:
a = train.loc[train['그룹코드'] == 1, ]
a.shape

In [ ]:
################################################################################

In [ ]:
# K-Fold 지정
K = 3

# 사용하고자 하는 *모든* 변수를 입력하세요
all_feature = ['노출(분)', '상품군', '판매단가',
'월', '일', '시간hour',
'계절', '168시간', '휴일', '방송시간', '매진여부',
'sale단어', '결제수단', '한달_상품군빈도',
'한달_중분류빈도', '한달_소분류빈도', '한달_그룹코드빈도', '기온', '강수량', '풍속', '습도', '적설',
'전운량', '비눈여부_평균이상', '미세먼지', '초미세먼지', '판매단가순위', '상품군판매단가순위',
'상품군요일순위', '상품군24시간순위', '중분류요일순위', '중분류24시간순위', '상품군168시간순위',
'중분류168시간순위', '판매단가rank', '시간순위', '시간순위168', '브랜드순위', '중분류순위', '소분류순위',
'방송내_순서', '방송순서', '판매상품종류수', '그룹코드_전체횟수', '그룹코드_대박횟수',
'그룹코드_대박확률', '브랜드별_전체횟수', '브랜드별_대박횟수', '브랜드별_대박확률', '중분류별_전체횟수',
'중분류별_대박횟수', '중분류별_대박확률', '소분류별_전체횟수', '소분류별_대박횟수', '소분류별_대박확률',
'월_COS', '월_SIN', '168시간_COS', '168시간_SIN', '24시간_COS', '24시간_SIN']

# PCA를 수행하고자 하는 변수를 입력하세요
pca_feature = []

# Target 변수를 입력하세요 '취급액'으로 넣어도 됩니다.
target = ['취급액boxcox']

# all_feature에 입력했던 범주형 변수를 입력하세요
onehot_feature = ['계절', '휴일', '결제수단', '상품군', '매진여부']

# 파라매터를 입력하세요
params = {'learning_rate' : 0.1, 'num_iterations' : 1000, 'max_depth' : 6}

In [ ]:
# Validataion set

y_pred, y_real, temp = PCA_CV(K, train, params, all_feature, pca_feature, target, onehot_feature)


In [ ]:
# test set : 2019.06 예측

y_pred, y_real, mape = final_predict(train, test, params, all_feature, pca_feature, target, onehot_feature)